## Install the theos python package

In [1]:
!sudo apt install tesseract-ocr && wget https://bootstrap.pypa.io/get-pip.py && python3.9 get-pip.py && python3.9 -m pip install --upgrade pip && python3.9 -m pip install theos-ai[ocr]==0.0.28

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
--2023-04-09 14:54:29--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2574273 (2.5M) [text/x-python]
Saving to: ‘get-pip.py.1’

get-pip.py.1        100%[===================>]   2.45M  --.-KB/s    in 0.01s   

2023-04-09 14:54:29 (213 MB/s) - ‘get-pip.py.1’ saved [2574273/2574273]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Succ

## Run the setup to finish installing theos

In [3]:
!theos setup --version v1 --subfield object-detection

[theos] setting up object-detection subfield...

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.4.1.tar.gz (24.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-1.11.0-cp39-cp39-manylinux1_x86_64.whl (750.6 MB)
  Using cached torchvision-0.12.0-cp39-cp39-manylinux1_x86_64.whl (21.0 MB)
  Using cached seaborn-0.11.0-py3-none-any.whl (283 kB)
  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3

## Restart the colab runtime

Click the restart runtime option on the top menu.

<img src="https://drive.google.com/uc?id=127ow4FbCF8cfBVfC-bWdhsv4l0-Usb2m" width="650px"/>


## Log in to account

In [3]:
!theos login


 Theos CLI

[*] username: rahulkarmakar
[*] password: 

[theos] login succeeded


# Set our theos project


In [4]:
!theos set-project 252539db-5f84-4228-96cd-bb8fc0af6a76

[theos] project added successfully


# Install the algorithm and weights


In [5]:
!theos install yolov7 --version tiny --weights license-plate-detection:experiment-2:best

# Detect and read

In [6]:
ALGORITHM = 'yolov7' # set the algorithm, such as 'yolov7'
ALGORITHM_VERSION = 'tiny' # set the algorithm version, such as 'tiny'
WEIGHTS = 'license-plate-detection:experiment-2:best' # set the weights tag, such as 'license-plates:experiment-1:best'

OCR_MODEL_SIZE = 'large' # set the OCR model size, possible values are 'small', 'medium' and 'large'
OCR_MODEL_TYPE = 'printed' # set the OCR model type if using OCR_MODEL_SIZE = 'large', possible values are 'str', 'printed' and 'handwritten'
OCR_MODEL_ACCURACY = 'best' # set the OCR model accuracy if using OCR_MODEL_SIZE = 'large', possible values are 'base', 'medium' and 'best'
OCR_CLASSES = ['license_plate'] # set the class names that the OCR model should read from, such as ['license-plate']

INPUT_IMAGE = 'image.jpg'

# Google Sheet id and Worksheet name

In [7]:
sheet_id = "1BhmkkB7uRV2sRoS7QGmrFqvFocnuF8ZNWHpZlAmuJyc"
worksheet_name = "Sheet1"

# Installing dependencies for Google Sheet

In [8]:
!pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import pandas as pd

In [11]:
from google.colab import drive

# Mount with Google Drive

In [12]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from google.colab import auth

In [14]:
auth.authenticate_user()

In [15]:
import gspread

In [16]:
from google.auth import default

In [17]:
creds, _ = default()

In [18]:
gc = gspread.authorize(creds)

In [19]:
sh = gc.open_by_key(sheet_id)

In [20]:
sh.worksheets()

[<Worksheet 'Sheet1' id:0>]

In [21]:
worksheet = sh.worksheet(worksheet_name)

# Actual detection and convert RC number into String

In [71]:
from theos.computer_vision.object_detection.utils import draw
from theos.computer_vision import ocr
from theos.client import Client
from google.colab import files
from tqdm import tqdm
import numpy as np
import torch
import json
import time
import cv2
import os

if torch.cuda.is_available():
  torch.cuda.empty_cache()

client = Client(inputs='.', outputs='.')
yolov7 = client.get(ALGORITHM, version=ALGORITHM_VERSION)
yolov7.load_weights(WEIGHTS)
yolov7.to_gpu()

if OCR_MODEL_TYPE and OCR_MODEL_ACCURACY: 
  ocr_model = ocr.load(size=OCR_MODEL_SIZE, model_type=OCR_MODEL_TYPE, accuracy=OCR_MODEL_ACCURACY)
else:
  ocr_model = ocr.load(size=OCR_MODEL_SIZE)

if OCR_MODEL_SIZE == 'large':
  ocr_model.to_gpu()

image = cv2.imread(INPUT_IMAGE)


detections = yolov7.detect(image)
detections = ocr_model.read(image, detections=detections, classes=OCR_CLASSES)
detected_image = draw(image, detections, alpha=0.15)
cv2.imwrite('detected_ocr.jpg', detected_image)
#print(json.dumps(detections, indent=4))
for d in detections:
    text = d['text']
print(text)

[theos] importing yolov7 code...
[theos] yolov7 code imported successfully
[theos] loading yolov7...
[theos] loading the large OCR model...
[theos] large OCR model loaded successfully
KL54-A-2670


# Read and Update Google Sheet

In [72]:
df = pd.DataFrame(worksheet.get_all_records())

In [73]:
df

,Date,Time,Key,Reg_No
0,4/9/2023,21:23:14,73233209208,MH:20 DV 2363
1,4/9/2023,21:25:56,14614420530,DL 7CQ 1939
2,4/9/2023,21:29:36,21813121252,IN59A Q 15 15
3,4/9/2023,21:34:03,6619913457,IN59AQ1515
4,4/9/2023,21:37:21,6619913457,


In [74]:
rows_to_update = df[df['Reg_No'] == ""]

In [75]:
for row in rows_to_update.itertuples(index = True, name = 'Pandas'):
  df.at[row.Index, 'Reg_No'] = text.replace(" ","")

In [76]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1BhmkkB7uRV2sRoS7QGmrFqvFocnuF8ZNWHpZlAmuJyc',
 'updatedRange': 'Sheet1!A1:D6',
 'updatedRows': 6,
 'updatedColumns': 4,
 'updatedCells': 24}